In [3]:
from dash import Dash, html, dcc, callback, Output, Input, State
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table
import plotly_express as px
import pandas as pd

In [5]:

os_data = pd.read_csv("data/athlete_events.csv")
os_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/athlete_events.csv'

In [ ]:
app = Dash(
    # får namnet på filen
    __name__, 
    # QUARTZ ändras till det temat man vill ha på bakgrundsfärg tex. Se på https://bootswatch.com/quartz/
    external_stylesheets=[dbc.themes.QUARTZ],
    # Säger till webbläsaren att siten är responsive (ändrar sig på mindre enheter)
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            # Skapar en rubrik med angivet textinnehåll, classname? mt-3 gör att det blir marginal ovanför rubriken på siten, "text-center" centrerar innehållet i kolumnen. "text-primary" i parantesen om väla förvald färg, "primary" byts ut mot den färgen man vill använda. Länk för förvalda färger: https://bootswatch.com/darkly/
            html.H1("Olympics results for Germany", className="text-center mt-3", style={"color":"#FFFFFF", "font-family": "Livre Baskerville"})
            # width= Kolumnen sträcker sig över hela skärmen.Här ändras with för att bestämma hur presenteras på mobilen för large, medium och smal mobiler osv.
        ], width=12)
    ]), 

    html.Br(),
    html.Br(),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                 # "id" för att ha en referens när man kopplar funktion/callbacks (?)
                 id="dropdown1",
                 #False gör att man endast kan välja ett val åt gången. True flera val.  
                 multi=False,
                 options=[data for data in os_data["Sport"].unique()], # Uppdaterad struktur för options
                 # Hexadecimal RGB står färgkoden för. Ändra färg? Googla? Snyggare lista typ lite dimmad eller något? Länkar för välja färg + konvertera till hex. https://www.rapidtables.com/web/color/RGB_Color.html , https://www.rgbtohex.net/
                 style={"color":"#144394"}    
            ),

            #Gör ett mellanrum per kod-rad. Annan kod som man kan välja storleken på bredden istället?
            html.Br(),
            html.Br(),

            # Skapar ett diagram/graf. Id används för att koppla en funktion till den sedan. Om jag använder detta diagrammet i en callback kan jag bara göra tomma måsvingar och ta bort diagram koden här för callbacksen körs alltid först ändå.
            dcc.Graph(
                id="Test_diagram1",
                figure=px.histogram(
                    os_data,
                    x="NOC",
                    y="Age",
                    title="Test Graph with NOC and Ages (original dataset)", 
                    template='plotly_white',
                    labels={"NOC": "NOC", "Age": "Ages"},
                    color="NOC"
                )
            )
                
            # Bredden på grafen. Halva sidan är width= 6 exempelvis.
        ], width=6),  # Ta bort stängande parentes här

        dbc.Col([
            dcc.Dropdown(
                 # "id" för att ha en referens när man kopplar funktion/callbacks (?)
                 id="dropdown2", 
                 multi=True,
                 options=[data for data in os_data["Sport"].unique()], # Uppdaterad struktur för options
                 # Hexadecimal RGB står färgkoden för. Ändra färg? Googla? Snyggare lista typ lite dimmad eller något? Länkar för välja färg + konvertera till hex. https://www.rapidtables.com/web/color/RGB_Color.html , https://www.rgbtohex.net/
                 style={"color":"#144394"}    
            ),

            html.Br(),
            html.Br(),

            dcc.Graph(
                id="Test_diagram2",
                figure=px.histogram(
                    os_data,
                    x="NOC",
                    y="Age",
                    title="Test Graph with NOC and Ages (original dataset)", 
                    template='plotly_white',
                    labels={"NOC": "NOC", "Age": "Ages"},
                    color="NOC"
                )
            )
        ], width=6)
    ], justify="evenly"),

    html.Br(),
    html.Br(),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                 # "id" för att ha en referens när man kopplar funktion/callbacks (?)
                 id="dropdown3", 
                 multi=False,
                 options=[data for data in os_data["Sport"].unique()], # Uppdaterad struktur för options
                 # Hexadecimal RGB står färgkoden för. Ändra färg? Googla? Snyggare lista typ lite dimmad eller något? Länkar för välja färg + konvertera till hex. https://www.rapidtables.com/web/color/RGB_Color.html , https://www.rgbtohex.net/
                 style={"color":"#144394"}    
            ),

            html.Br(),
            html.Br(),

            # Skapar ett diagram/graf. Id används för att koppla en funktion till den sedan. Om jag använder detta diagrammet i en callback kan jag bara göra tomma måsvingar och ta bort diagram koden här för callbacksen körs alltid först ändå.
            dcc.Graph(
                id="Test_diagram3",
                figure=px.histogram(
                    os_data,
                    x="NOC",
                    y="Age",
                    title="Test Graph with NOC and Ages (original dataset)", 
                    template='plotly_white',
                    labels={"NOC": "NOC", "Age": "Ages"},
                    color="NOC"
                )
            )
                
            # Bredden på grafen. Halva sidan är width= 6 exempelvis.
        ], width=6),  # Lägg till en avslutande parentes här

        dbc.Col([
            dcc.Dropdown(
                 # "id" för att ha en referens när man kopplar funktion/callbacks (?)
                 id="dropdown4", 
                 multi=False,
                 options=[data for data in os_data["Sport"].unique()], # Uppdaterad struktur för options
                 # Hexadecimal RGB står färgkoden för. Ändra färg? Googla? Snyggare lista typ lite dimmad eller något? Länkar för välja färg + konvertera till hex. https://www.rapidtables.com/web/color/RGB_Color.html , https://www.rgbtohex.net/
                 style={"color":"#144394"}    
            ),

            html.Br(),
            html.Br(),

            # Skapar ett diagram/graf. Id används för att koppla en funktion till den sedan. Om jag använder detta diagrammet i en callback kan jag bara göra tomma måsvingar och ta bort diagram koden här för callbacksen körs alltid först ändå.
            dcc.Graph(
                id="Test_diagram4",
                figure=px.histogram(
                    os_data,
                    x="NOC",
                    y="Age",
                    title="Test Graph with NOC and Ages (original dataset)", 
                    template='plotly_white',
                    labels={"NOC": "NOC", "Age": "Ages"},
                    color="NOC"
                )
            )
                
            # Bredden på grafen. Halva sidan är width= 6 exempelvis.
        ], width=6)  # Ta bort stängande parentes här
    ])

], fluid=True)






# #Används för att koppla samman något i layouten med funktionen.
# @callback(
#     #I outputens första attribut (ex: Test_diagram) skall id:t stå som föremålet i layouten har. Samt andra attributet: namnet på grafen.
#     Output("Test_diagram", "figure"),
#     #Input är dropdown listan som har id:t som är döpt till "dropdown1", samt "value" kan man sätta till ett värde om man vill att listan ska vara startinställd på något. Hur lägger man till default? Finns i en video. 
#     Input("dropdown1", "value")
# )

# #Skapar en funktion till det som är skapat i layouten. Alltså funktionen retunerar ditt villkor. Callback säger vad som påverkar vad. Dvs när du ändrar i dropdownen "Input", påverkat diagrammet "output".

# def graf_function1():
#     #Retunerar ett nytt diagram, samma kod för diagrammet som står ovan. Nu retunerar den bara det diagrammet med egenskaperna i funktionen. Skriv egna villkor.
#     return px.histogram(os_data,x="NOC", y="Age")



# Debug=False när appen är klar.
app.run(debug=True, jupyter_mode='external', port=300)